In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime as dt

import numpy as np
import pandas as pd

from deep_hedging import (
    Underlyings,
    Ticker,
    ConstantRateCurve,
    EuropeanCall,
    EuropeanPut,
)
from deep_hedging.monte_carlo import GBMPricer

RANDOM_SEED = 12

In [3]:
N_PATHS = 20_000
N_STOCKS = 1
TILL_MATURITY = 3.0
N_DAYS = 365 * TILL_MATURITY
RF_RATE = 0.03
VOL = 0.2

In [4]:
N_DAYS

1095.0

In [5]:
np.random.seed(RANDOM_SEED)

start = dt.datetime(2019, 1, 1)
end = start + dt.timedelta(days=N_DAYS)

data = pd.DataFrame(
    np.concatenate(
        [
            np.expand_dims(np.array([1.0]), 1),
            np.exp(
                RF_RATE / 252
                + VOL / np.sqrt(252) * np.random.randn((end - start).days, N_STOCKS)
            ),
        ],
        axis=0,
    ),
    columns=["price"],
)
data["price"] = data["price"].cumprod()
data["index"] = pd.date_range(start=start, end=end, freq="1D")
data.set_index("index", inplace=True)

underlyings = Underlyings(
    tickers=[Ticker(f"Stock {i + 1}", f"{i + 1}") for i in range(N_STOCKS)],
    start=start,
    end=end,
    data=data,
    dividends=np.array([0.0] * N_STOCKS),
)
underlyings.data

,price
index,
2019-01-01,1.000000
2019-01-02,1.006097
2019-01-03,0.997615
2019-01-04,1.000786
2019-01-05,0.979686
...,...
2021-12-27,1.068472
2021-12-28,1.080112
2021-12-29,1.074025


In [6]:
np.sqrt(np.diag(underlyings.get_var_covar()))

array([0.2051043])

In [7]:
curve = ConstantRateCurve(rate=RF_RATE)

In [19]:
european_call = EuropeanCall(
    underlyings=underlyings,
    yield_curve=curve,
    strike_level=1.0,
    start_date=start,
    end_date=end,
)

In [9]:
pricer = GBMPricer(payoff_function=european_call.payoff, random_seed=RANDOM_SEED)

In [10]:
paths = pricer.get_paths(
    current_spot=[1.0],
    time_till_maturity=TILL_MATURITY,
    risk_free_rate_fn=curve.get_rate,
    dividends_fn=lambda t: 0.0,
    var_covar_fn=lambda t: np.array([VOL**2]),
    n_paths=N_PATHS,
)
paths.shape

(100000, 757, 1)

In [11]:
from typing import Callable


def get_greek(spot: np.array, greek_fn: Callable[[np.array], np.array]) -> np.array:
    greek = []
    for day in range(spot.shape[1]):
        greek.append(list(greek_fn(spot=spot[:, day]).flatten()))
    return np.array(greek).T

In [12]:
deltas = get_greek(paths, european_call.delta)
deltas.shape

(100000, 757)

In [13]:
deltas.min(), deltas.max()

(7.176208220730748e-05, 0.9999999788347615)

In [14]:
def get_delta_hedge_pnl(
    spot: np.array, greek_fn: Callable[[np.array], np.array]
) -> np.array:
    delta_paths = get_greek(spot, european_call.delta)

In [22]:
european_call.price()

[0.4330127]
[0.08660254]


array([0.17899527])

In [16]:
final_points = np.log(paths[:, -1, :].squeeze(1))
final_points.mean() / 3, final_points.std() / np.sqrt(3)

(0.010110012826833657, 0.19966178768086412)

In [17]:
underlyings.get_dividends()

array([0.])